<a href="https://colab.research.google.com/github/zlerdworatawee/iclue-scripts/blob/main/anti_diag_final_var.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display, Math
import numpy as np
import cupy as cp # only for gpu use
import pandas as pd
import sympy as sp
from sympy import symbols, sympify, Matrix, Eq, solve, expand, Abs, pprint, factor
from itertools import permutations
import scipy
import sys
import math

# Anti-Diagonal Code RSK Variant

> Functions below


In [ ]:
def contingency_tables(row_sums: np.array, col_sums: np.array) -> list:
    '''
    This function generates the contingency tables
    for a given set of row and column sums
    using a backtracking/recursive approach.
    '''
    m, n = len(row_sums), len(col_sums)
    result = []

    def backtrack(row, table, col_sums_left):
        if row == m:
            if all(c == 0 for c in col_sums_left):
                result.append(np.array(table))
            return

        def valid_next_rows(rsum, cols_left, partial=[]):
            if len(partial) == n:
                if sum(partial) == rsum:
                    yield list(partial)
                return
            i = len(partial)
            max_entry = min(rsum - sum(partial), cols_left[i])
            for x in range(max_entry + 1):
                yield from valid_next_rows(rsum, cols_left, partial + [x])

        for row_vals in valid_next_rows(row_sums[row], col_sums_left):
            new_cols = [c - x for c, x in zip(col_sums_left, row_vals)]
            if all(c >= 0 for c in new_cols):
                backtrack(row + 1, table + [row_vals], new_cols)

    backtrack(0, [], col_sums)
    return result

In [ ]:
def rsk_insert(tableau, x):
    tableau = tableau.copy()
    rows, cols = tableau.shape
    bumped = x
    for r in range(rows):
        row = tableau[r]
        mask = (row > 0)
        eligible = row[mask]
        idx = np.where(eligible > bumped)[0]
        if idx.size == 0:
            insert_pos = np.sum(mask)
            if insert_pos < cols:
                tableau[r, insert_pos] = bumped
                return tableau, (r, insert_pos)
            else:
                continue
        else:
            i = idx[0]
            bumped, tableau[r, i] = tableau[r, i], bumped
    empty_row = np.zeros(cols, dtype=int)
    empty_row[0] = bumped
    tableau = np.vstack([tableau, empty_row])
    return tableau, (tableau.shape[0] - 1, 0)

def viennot_rsk(biword):
    n = len(biword)
    P = np.zeros((n, n), dtype=int)
    Q = np.zeros((n, n), dtype=int)
    for a, b in biword:
        P, (r, c) = rsk_insert(P, a)
        Q[r, c] = b
    return P, Q

def print_tableau(tableau, name='T'):
    '''
    for debugging
    @params tableau is a numpy array
    @params name is a string
    '''
    print(f"{name}:")
    for row in tableau:
        row_nonzero = row[row > 0]
        if row_nonzero.size > 0:
            print(row_nonzero)

In [ ]:
def contingency_vectors(n, d):
    '''
    This function generates the possible contingency vectors
    @params n: len of vector
    @params d: degree
    '''
    vectors = []
    def recusion(vector, degrees_left):
        if len(vector) == n:
            vectors.append(vector)
            return
        for i in range(degrees_left + 1)[::-1]:
            new_vector = vector + [i]

            if len(new_vector) == n:  # pruning
                vectors.append(new_vector)
                return

            recusion(new_vector, degrees_left - int(i))

    recusion([], d)
    return vectors

# Anti-Diagonal RSK Check Code



In [ ]:
def longest_antidiag(M, i):
  '''finds the longest anti-diagonal ending in specified row (SW to NE) i given a matrix M using dynamic programming
  Args:
    - M: generic np.array
    - i: given last row of the anti-diagonal
  '''
  if i < 0 or i >= M.shape[0]:
      return 0

  col_indexes = np.where(M[i] != 0)[0]  # non-zero cols in row i
  if len(col_indexes) == 0:
      return longest_antidiag(M, i - 1)

  longest = 1
  for col_idx in col_indexes:
      M_take = M[:i, col_idx + 1:]     # rows above, cols to the right
      longest = max(longest, 1 + longest_antidiag(M_take, i - 1))

  return longest

In [ ]:
def lowest_row_leq_i(P, i) -> int:
    '''Finds lowest row in tableau P containing some nonzero j <= i'''
    try:
        rows, _ = np.where((P <= i) & (P != 0))
        if len(rows) == 0:
            return math.inf
        return np.max(rows) + 1  # +1 if you want 1-based indexing
    except:
        return math.inf

In [ ]:
def lowest_row_with_i(P, i) -> int:
  '''finds lowest row in tableau P containing i
  Args:
    - P: generic 2D np.array
    - i: given number
  '''
  try:
      row = np.max(np.where(P == i)[0])
      return row + 1
  except:
      return math.inf

In [ ]:
def biword(M: np.array) -> list:
  '''given a matrix M return the corresponding biword
  Args:
    - M: generic np.array
  '''
  # biword goes from left to right col-wise and top to bottom row-wise
  biword_P = []
  for j in range(M.shape[1]):
    for i in range(M.shape[0]):
        if M[i, j] != 0:
            biword_P.extend([i + 1] * M[i, j])
  biword_Q = np.arange(1, len(biword_P) + 1, 1) # recording tableau Q doesn't matter so this is a dummy value
  return list(zip(biword_P, biword_Q))

# End of Functions
> begining of testbed

testing_anti_diag() tests a large amount of matrices

testing_singular_matrix() tests just one but you have to manually change test_matrix

In [ ]:
def test_singular_matrix(input_matrix: np.array) -> int:
  '''Testing a single matrix
  filler fxn for testbed'''
  test_matrix = input_matrix
  n = len(test_matrix[0])
  bw = biword(test_matrix)
  P = viennot_rsk(bw)[0]
  i_vals = np.delete(np.unique(P), np.where(np.unique(P) == 0))
  for i in i_vals:
    low_row_eq = lowest_row_with_i(P, i)
    longest_antidiag_ending_i = longest_antidiag(test_matrix, i - 1)
    low_row_leq = lowest_row_leq_i(P, i)
    if low_row_eq <= longest_antidiag_ending_i and longest_antidiag_ending_i <= low_row_leq:
       print(f"Checking: {low_row_eq} <= {longest_antidiag_ending_i} <= {low_row_leq} → "
             f"{low_row_eq <= longest_antidiag_ending_i and longest_antidiag_ending_i <= low_row_leq}")
       continue
    else:
       print(f"""SANITY CHECK FAILED:
             P: {P}
             i: {i}
             lowest_row_with_i: {low_row_eq}
             longest_antidiag_ending_i: {longest_antidiag_ending_i}
             lowest_row_leq_i: {low_row_leq}
             """)
       return -1
  print("SANITY CHECK PASSED")
  return 1

In [ ]:
def comprehensive_test():
    ''' easy testing function for sanity checking the following:
    (Lowest row in P containing i)
    <= (Longest antidiagonal in M ending in row i)
    <= (Lowest row in P containing some j <= i)
    for matrices with total sum <= (SOME NUMBER)

    !!!caution: this will take a long time to run since it goes through
    all possible matrices with given total and size and then tests them
    for each distinct value in each matrix!!!
    '''
    while True:
        try:
            matrix_total_sum = int(input("Enter the total sum of the matrix: ")) # SOME_NUMBER
            matrix_size = int(input("Enter the size of the (square) matrix: "))
            break
        except ValueError:
            print("Invalid input. Please enter integers.")
    matrices = []
    P_tableaus = []

    for i in range(matrix_total_sum):
      contingency_vecs = contingency_vectors(matrix_size, i + 1)
      for pi in contingency_vecs:
          for sig in contingency_vecs:
              matrices.extend(contingency_tables(pi, sig))

    for matrix in matrices:
      if test_singular_matrix(np.array(matrix)) == -1:
        print(matrix)
        break
    print("TESTING COMPLETE")

In [ ]:
comprehensive_test()

KeyboardInterrupt: Interrupted by user

In [ ]:
def test_singular_matrix_manual():
  '''Testing a single matrix
  there's no good way for me to have you input a matrix so you have to edit the one below'''
  test_matrix = np.array([
    [0, 1, 1, 0],
    [1, 2, 0, 0],
    [0, 1, 0, 1],
    [1, 0, 2, 0]
  ])
  n = len(test_matrix[0])
  bw = biword(test_matrix)
  P = viennot_rsk(bw)[0]
  i_vals = np.delete(np.unique(P), np.where(np.unique(P) == 0))
  print(i_vals)
  for i in i_vals:
    low_row_eq = lowest_row_with_i(P, i)
    longest_antidiag_ending_i = longest_antidiag(test_matrix, i - 1)    # python zero indexing
    low_row_leq = lowest_row_leq_i(P, i)
    if low_row_eq <= longest_antidiag_ending_i and longest_antidiag_ending_i <= low_row_leq:
      print(f"Checking: {low_row_eq} <= {longest_antidiag_ending_i} <= {low_row_leq} → "
             f"{low_row_eq <= longest_antidiag_ending_i and longest_antidiag_ending_i <= low_row_leq}")
      continue
    else:
       print(f"""SANITY CHECK FAILED:
             M: {test_matrix}
             P: {P}
             i: {i}
             lowest_row_with_i: {low_row_eq}
             longest_antidiag_ending_i: {longest_antidiag_ending_i}
             lowest_row_leq_i: {low_row_leq}
             """)
       return -1
  return 1

In [ ]:
test_singular_matrix_manual()

[1 2 3 4]
Checking: 1 <= 1 <= 1 → True
Checking: 2 <= 2 <= 2 → True
Checking: 1 <= 2 <= 2 → True
Checking: 3 <= 3 <= 3 → True


1